In [1]:
# Optimize LightGBM hyperparameters using Optuna with cross-validation
import optuna
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from torch_geometric.utils import from_smiles
import pandas as pd
from pyg_chemprop_utils import smiles2data


dataset_clean = pd.read_csv("dataset/SMRT_dataset_with_dates.csv")
# Randomly split the dataset into training and validation sets
train_set, valid_set = train_test_split(dataset_clean, test_size=0.2, random_state=42)

# Build graph dataset from train/valid splits
smiles_col = "SMILES"  # change if your column name differs
rt_col = "rt"          # change if your column name differs

train_graphs = []
for _, row in train_set.iterrows():
    data = smiles2data(row[smiles_col])
    data['y'] = row[rt_col]
    if data is not None:
        train_graphs.append(data)

valid_graphs = []
for _, row in valid_set.iterrows():
    data = smiles2data(row[smiles_col])
    data['y'] = row[rt_col]
    if data is not None:
        valid_graphs.append(data)

len(train_graphs), len(valid_graphs)

(64030, 16008)

In [2]:
from torch_geometric.data import InMemoryDataset
import torch

class MyDataset(InMemoryDataset):
    def __init__(self, data_list, transform=None):
        super().__init__(".", transform)
        self.data, self.slices = self.collate(data_list)

    def _download(self):
        pass

    def _process(self):
        pass

In [71]:
from pyg_chemprop import DMPNNEncoder, RevIndexedData

train_graphs = [RevIndexedData(graph) for graph in train_graphs]
valid_graphs = [RevIndexedData(graph) for graph in valid_graphs]

In [72]:
train_graphs[0]

RevIndexedData(y=865.2, edge_index=[2, 104], x=[49, 133], edge_attr=[104, 14], revedge_index=[104])

In [73]:
from torch_geometric.loader import DataLoader
batch_size=512

train_loader = DataLoader(train_graphs, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_graphs, batch_size=batch_size, shuffle=False)

In [74]:
from tqdm import tqdm
from pyg_chemprop_utils import initialize_weights
def train(config, loader, device=torch.device("cpu")):
    criterion = config["loss"]
    model = config["model"]
    optimizer = config["optimizer"]
    scheduler = config["scheduler"]

    model = model.to(device)
    model.train()
    for batch in tqdm(loader, total=len(loader)):
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch)
        loss = criterion(out.squeeze(), batch.y.float())
        loss.backward()
        optimizer.step()
        scheduler.step()
def make_prediction(config, loader, device=torch.device("cpu")):
    model = config["model"]

    model = model.to(device)
    model.eval()
    y_pred = []
    y_true = []
    for batch in tqdm(loader, total=len(loader)):
        batch = batch.to(device)
        with torch.no_grad():
            batch_preds = model(batch)
        y_pred.extend(batch_preds)
        y_true.extend(batch.y)
    return torch.stack(y_pred).cpu(), torch.stack(y_true).cpu()

In [75]:
num_epochs = 50
hidden_size = 256
depth = 3
out_dim = 1

In [76]:
from torch import nn
head = nn.Sequential(
    nn.Linear(hidden_size, hidden_size, bias=True),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size//2, bias=True),
    nn.ReLU(),
    nn.Linear(hidden_size//2, out_dim, bias=True),
)
model = nn.Sequential(
    DMPNNEncoder(
        hidden_size,
        train_loader.dataset[0].num_node_features,
        train_loader.dataset[0].num_edge_features,
        depth,
    ),
    head,
)
initialize_weights(model)

In [77]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=1e-3, steps_per_epoch=len(train_loader), epochs=num_epochs
)
config = {
    "loss": criterion,
    "model": model,
    "optimizer": optimizer,
    "scheduler": scheduler,
}


In [57]:
y_true, y_pred.squeeze()

(tensor([ 612.8000, 1065.5000, 1137.9000,  ...,  739.2000,  673.8000,
          954.4000]),
 tensor([775.3771, 790.3701, 862.6066,  ..., 777.1818, 767.7501, 792.4919]))

In [78]:
from sklearn.metrics import mean_squared_error, r2_score
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}")
    train(config, train_loader)
    y_pred, y_true = make_prediction(config, valid_loader)
    mse = mean_squared_error(y_true, y_pred.squeeze())
    r2 = r2_score(y_true, y_pred.squeeze())
    print(f"val r2={r2:.6} mse={mse:.6}")

Epoch 1


100%|██████████| 32/32 [00:04<00:00,  7.72it/s]


val r2=-12.3121 mse=5.65318e+05
Epoch 2


100%|██████████| 32/32 [00:03<00:00,  8.23it/s]


val r2=0.00687665 mse=42174.5
Epoch 3


...

val r2=0.731706 mse=11393.5
Epoch 49


100%|██████████| 32/32 [00:04<00:00,  7.42it/s]


val r2=0.7317 mse=11393.8
Epoch 50


100%|██████████| 32/32 [00:04<00:00,  7.84it/s]


val r2=0.731699 mse=11393.8


In [80]:
y_pred, y_true = make_prediction(config, train_loader)
mse = mean_squared_error(y_true, y_pred.squeeze())
r2 = r2_score(y_true, y_pred.squeeze())
print(f"test r2={r2:.6} mse={mse:.6}")

100%|██████████| 126/126 [00:17<00:00,  7.29it/s]


test r2=0.740261 mse=11110.7
